In [ ]:
pip install pyAudioAnalysis


python(39704) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 8.8 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
  Created wheel for pyAudioAnalysis: filename=pyaudioanalysis-0.3.14-py3-none-any.whl size=41264435 sha256=35458999cb3eb415866f400b13ba42ee37cfb69b9b9c5138239f4a3035e389c2
  Stored in directory: /Users/byeon-ujung/Library/Caches/pip/wheels/24/9e/ab/d18e8a5866e4b4edfc012d6576e5ed699dd2aa4acbab8e8c90
Successfully built pyAudioAnalysis

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import subprocess
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from urllib.parse import urlparse, parse_qs

# === 경로 설정 ===
SAVE_DIR = "temp_audio"
OUTPUT_FILE = "audio_features_with_errors.csv"ㅁ
os.makedirs(SAVE_DIR, exist_ok=True)

# === video_id 추출 함수 ===
def extract_video_id(url):
    parsed = urlparse(url)
    return parse_qs(parsed.query).get("v", [None])[0]

# === 원본 데이터 불러오기 ===
df = pd.read_csv("raw_data/youtube_info/비공개영상까지삭제(최종전)/final_all_youtubeAPI_info.csv", encoding="utf-8")
video_data = [
    {"video_id": extract_video_id(url), "url": url}
    for url in df['video_url'] if extract_video_id(url)
]

# === 이전에 처리된 결과 불러오기 ===
if os.path.exists(OUTPUT_FILE):
    df_done = pd.read_csv(OUTPUT_FILE)
    processed_ids = set(df_done["video_id"])
else:
    df_done = pd.DataFrame()
    processed_ids = set()

# === mp3 경로 반환 ===
def get_audio_path(video_id):
    return os.path.join(SAVE_DIR, f"{video_id}.mp3")

# === 영상 다운로드 + 분석 ===
def process_video(video_id, url):
    result = {
        "video_id": video_id,
        "url": url,
        "pitch_mean": None,
        "energy_mean": None,
        "centroid_mean": None,
        "bpm": None,
        "speech_rate": None,
        "initial_silence": None,
        "error": None
    }

    try:
        # --- MP3 다운로드 ---
        output_path = get_audio_path(video_id)
        command = [
            "yt-dlp",
            f"https://www.youtube.com/watch?v={video_id}",
            "--extract-audio",
            "--audio-format", "mp3",
            "--external-downloader", "ffmpeg",
            "--external-downloader-args", "-ss 0 -t 15",
            "-o", output_path
        ]
        subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        if not os.path.exists(output_path):
            raise FileNotFoundError("MP3 파일 생성 실패")

        # --- 오디오 분석 ---
        y, sr = librosa.load(output_path, sr=None)

        f0, _, _ = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        result["pitch_mean"] = np.nanmean(f0)

        rms = librosa.feature.rms(y=y)
        result["energy_mean"] = np.mean(rms)

        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        result["centroid_mean"] = np.mean(centroid)

        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        result["bpm"] = tempo

        onsets = librosa.onset.onset_detect(y=y, sr=sr, units='time')
        result["speech_rate"] = len(onsets) * (60.0 / 15.0)

        rms_vals = rms[0]
        result["initial_silence"] = librosa.frames_to_time(np.argmax(rms_vals > 0.01), sr=sr) if np.any(rms_vals > 0.01) else 15.0

    except Exception as e:
        result["error"] = str(e)

    finally:
        # MP3 삭제
        try:
            os.remove(output_path)
        except:
            pass

    return result

# === 분석 실행 루프 ===
with open(OUTPUT_FILE, "a", encoding="utf-8", newline='') as f_append:
    for item in tqdm(video_data, desc="Processing videos"):
        if item["video_id"] in processed_ids:
            continue

        row = process_video(item["video_id"], item["url"])
        df_temp = pd.DataFrame([row])
        df_temp.to_csv(f_append, header=(f_append.tell() == 0), index=False, float_format="%.4f")
        f_append.flush()
        processed_ids.add(item["video_id"])


Processing videos:   0%|          | 0/6411 [00:00<?, ?it/s]

python(40471) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40498) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40522) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40548) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40574) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40598) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40622) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40646) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40676) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40699) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40723) Malloc

In [ ]:
# 09:56 -> 22개
# 10:07 -> 66개
# 10:22 -> 131개

# -> 9분 -> 44개
# -> 26분 -> 109개

### error 뜬 영상들에 대해 재추출

In [ ]:
raw_df = pd.read_csv('raw_data/audio_features_with_errors.csv', encoding='utf-8')
raw_df

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
0,s6suc9Xz0rM,https://www.youtube.com/watch?v=s6suc9Xz0rM,79.6160,0.1234,1168.3195,[61.81318681],152.0,0.3627,NaN
1,kERWAGtltBU,https://www.youtube.com/watch?v=kERWAGtltBU,138.9813,0.1011,1351.3056,[83.95522388],124.0,0.5013,NaN
2,8VcAaEfuc0s,https://www.youtube.com/watch?v=8VcAaEfuc0s,89.3799,0.1323,862.8627,[108.17307692],176.0,0.3733,NaN
3,CI331OcJ8Do,https://www.youtube.com/watch?v=CI331OcJ8Do,176.4752,0.0737,1687.4316,[160.71428571],208.0,0.5013,NaN
4,AOrtPpF1t7U,https://www.youtube.com/watch?v=AOrtPpF1t7U,113.2603,0.0968,924.5371,[77.05479452],180.0,0.5547,NaN
...,...,...,...,...,...,...,...,...,...
6406,fLNAk4NYOJU,https://www.youtube.com/watch?v=fLNAk4NYOJU,91.7625,0.1175,1180.3476,[234.375],240.0,0.1600,NaN
6407,Sh41rnVFlhU,https://www.youtube.com/watch?v=Sh41rnVFlhU,109.4980,0.2931,2079.7808,[156.25],424.0,0.4693,NaN
6408,z1UGEkHIoCY,https://www.youtube.com/watch?v=z1UGEkHIoCY,103.9737,0.0451,343.8384,[119.68085106],60.0,0.3733,NaN
6409,xVaTzgcJttI,https://www.youtube.com/watch?v=xVaTzgcJttI,136.2584,0.1755,1891.7688,[144.23076923],188.0,1.2480,NaN


In [ ]:
raw_df.loc[raw_df['error'].notnull(), 'video_id']

29      Mzpga5zM3gI
32      9vunx_Fzzdc
63      CbaOrIR20Uo
113     IcHeRHYBd6I
138     oOQ7Ahe8vmI
           ...     
6352    NXNOK_-IilY
6371    gAtOZTt8-BM
6383    OvodpGjSna8
6404    AbDJl2CTPLs
6410    WEkYWyi1gfA
Name: video_id, Length: 411, dtype: object

In [ ]:
import os
import subprocess
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from urllib.parse import urlparse, parse_qs

# === 경로 설정 ===
SAVE_DIR = "temp_audio"
INPUT_FILE = "raw_data/youtube_info/비공개영상까지삭제(최종전)/final_all_youtubeAPI_info.csv"
ORIG_RESULT_FILE = "raw_data/audio_features_with_errors.csv"
RETRY_OUTPUT_FILE = "audio_features_retry.csv"
os.makedirs(SAVE_DIR, exist_ok=True)

# === 유틸 함수 ===
def extract_video_id(url):
    parsed = urlparse(url)
    return parse_qs(parsed.query).get("v", [None])[0]

def get_audio_path(video_id):
    return os.path.join(SAVE_DIR, f"{video_id}.mp3")

def process_video(video_id, url):
    result = {
        "video_id": video_id,
        "url": url,
        "pitch_mean": None,
        "energy_mean": None,
        "centroid_mean": None,
        "bpm": None,
        "speech_rate": None,
        "initial_silence": None,
        "error": None
    }

    try:
        output_path = get_audio_path(video_id)
        command = [
            "yt-dlp",
            f"https://www.youtube.com/watch?v={video_id}",
            "--extract-audio",
            "--audio-format", "mp3",
            "--external-downloader", "ffmpeg",
            "--external-downloader-args", "-ss 0 -t 15",
            "-o", output_path
        ]
        subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        if not os.path.exists(output_path):
            raise FileNotFoundError("MP3 파일 생성 실패")

        y, sr = librosa.load(output_path, sr=None)
        f0, _, _ = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        result["pitch_mean"] = np.nanmean(f0)

        rms = librosa.feature.rms(y=y)
        result["energy_mean"] = np.mean(rms)

        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        result["centroid_mean"] = np.mean(centroid)

        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        result["bpm"] = tempo

        onsets = librosa.onset.onset_detect(y=y, sr=sr, units='time')
        result["speech_rate"] = len(onsets) * (60.0 / 15.0)

        rms_vals = rms[0]
        result["initial_silence"] = librosa.frames_to_time(np.argmax(rms_vals > 0.01), sr=sr) if np.any(rms_vals > 0.01) else 15.0

    except Exception as e:
        result["error"] = str(e)
    finally:
        try:
            os.remove(output_path)
        except:
            pass

    return result

# === 데이터 로딩 및 오류 ID 필터링 ===
df_info = pd.read_csv(INPUT_FILE, encoding="utf-8")
df_orig = pd.read_csv(ORIG_RESULT_FILE, encoding="utf-8")
error_ids = set(df_orig.loc[df_orig["error"].notnull(), "video_id"])

video_data = [
    {"video_id": extract_video_id(url), "url": url}
    for url in df_info["video_url"] if extract_video_id(url)
]
video_data_error_only = [v for v in video_data if v["video_id"] in error_ids]

# === 오류난 영상만 재처리 ===
results = []
for item in tqdm(video_data_error_only, desc="Retrying failed videos"):
    row = process_video(item["video_id"], item["url"])
    results.append(row)

df_retry = pd.DataFrame(results)
df_retry.to_csv(RETRY_OUTPUT_FILE, index=False, encoding="utf-8", float_format="%.4f")
print(f"🔄 오류 재처리 완료 → {FINAL_FIXED_FILE} 저장됨")


Retrying failed videos:   0%|          | 0/411 [00:00<?, ?it/s]

python(41510) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41552) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41593) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41633) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41671) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41702) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41734) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41765) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41798) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41820) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(41848) Malloc

🔄 오류 재처리 완료 → audio_features_with_errors_fixed.csv 저장됨


### 재처리에서 오류난 것 확인
* 87+5 = 92개 모두 삭제 처리 -> 6319개만 있음
* 오류나는 것 삭제할 것: ['aR4cXPWdTm0', 'VU5UIyUTKrk', 'GRHd6_gYzfw', '93d5yTcKzY0', '7ud88gf0P-8',
 '8lT4E63u2xg', 'RvCSkwSMEOE', 'wIlUEIWD_Iw', 'zlbnEdIdlZg', 'ThVzGu3-XPM',
 'J-M9khq5hLs', 'aLdthnK_1UQ', 'cFjAnSlYkmk', '3fQwMOb69o8', '29OWoBqY-eA',
 'RbUK3vkkR0U', 'Ep-nD7IeHnM', '3XMcC__sHnQ', 'DIaWb1ciHHY', 'nQLDqRSiF_k',
 'fbxz-lu56NE', 'urGtvYuMVYQ', 'Vwk7wY_HtmI', '7KTgTqde-FE', 'cKq36-nvC3E',
 '4huQ4Q7nRdM', '0TUPSJSMQdc', 'XlnwgHSsSdA', 'tTi67LVX3hU', 'EM6Karmdf7s',
 'NMmr0ukHEGo', 'D-FZ5OTw-NI', 'rgAEPHxrwgA', 'KsFcb9xs7eQ', 'yfLYCxeobbU',
 'GU22ga_7D2s', 'PWtMDf36wAA', 'ULwLE6L-wL4', 'iE_zT3vlDNE', 'a_Pv8PmlDMY',
 'EJUy5prRQks', 'faEIo-k2BZY', 'AE11rWYdHdQ', '7cyheohi0YU', 'ci_6i0T7o1M',
 'eGPuC9Z2riU', 'oX3VdftQzLQ', 'ajF2J-PzJWE', 'T5B-zcm5UO0', 'ucs7n403hPw',
 '84VFoLLqJn8', '6UoB6LVl0YY', 'vnYV3Twx7lU', 'M7tpUacxXyc', '8lfLS8ONB5g',
 'NEBIFMrJ7VI', 'UZVgD0Nf9Sg', 'Oa87_UgW57A', 'VcdwvmV8sTQ', '4j80w4_I3fk',
 'p5YhWhpl0ZI', 'Z8IvrKBPlew', 'UxIMzyPfnQ0', 'CXvDo7Sg800', 'JhNgb-yOxtE',
 'Qhe9OSA96uM', 'CfmTK-baSPU', '4hvWRsZJiag', '9CYxVsFu_dA', 'p4vqfrL_1ew',
 '2iV4-lS28tY', 'G-ie9Igk6qc', 'wBU1owueras', 'Y5wp8g1_Qlw', 'ZbNq-Pl5Rz8',
 'ezykDVItXFI', 'FT5RqC_S-xM', 'hejFoS8SlIQ', '0Q3nYDKOlF4', '_xjvovktwZw',
 'r40JmhK8Q0w', 'lhpYR74TCN0', 'bPRpGeFb4xM', 'zjwZ73U236I', 'JKBCcxRry-U',
 'Xc8nr_GQoZw', 'AbDJl2CTPLs']
* pitch_mean 이 NaN인 것 삭제할 것: ['q3FEbYsyy7M', '2FnF1WzQx58', 'J9DNUz52iY8', '4W1-eIJ_cnQ', 'olxg_Vbg7z0']

In [ ]:
retry1_df = pd.read_csv('raw_data/audio_features/audio_features_retry.csv', encoding='utf-8')
retry1_df

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
0,Mzpga5zM3gI,https://www.youtube.com/watch?v=Mzpga5zM3gI,104.8646,0.1190,2025.8531,[40.17857143],180.0,0.4800,NaN
1,9vunx_Fzzdc,https://www.youtube.com/watch?v=9vunx_Fzzdc,150.6638,0.1252,1381.3915,[112.5],208.0,0.5120,NaN
2,CbaOrIR20Uo,https://www.youtube.com/watch?v=CbaOrIR20Uo,109.1374,0.1690,638.5839,[43.9453125],256.0,0.6080,NaN
3,IcHeRHYBd6I,https://www.youtube.com/watch?v=IcHeRHYBd6I,91.7853,0.0664,1441.1983,[170.45454545],268.0,1.0667,NaN
4,oOQ7Ahe8vmI,https://www.youtube.com/watch?v=oOQ7Ahe8vmI,148.8142,0.0601,1373.6295,[127.84090909],212.0,1.3867,NaN
...,...,...,...,...,...,...,...,...,...
406,NXNOK_-IilY,https://www.youtube.com/watch?v=NXNOK_-IilY,86.2420,0.0619,929.9413,[127.84090909],144.0,4.0533,NaN
407,gAtOZTt8-BM,https://www.youtube.com/watch?v=gAtOZTt8-BM,118.8233,0.0700,1569.3830,[160.71428571],68.0,0.5227,NaN
408,OvodpGjSna8,https://www.youtube.com/watch?v=OvodpGjSna8,111.2943,0.0520,2463.7146,[114.79591837],72.0,0.7040,NaN
409,AbDJl2CTPLs,https://www.youtube.com/watch?v=AbDJl2CTPLs,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."


In [ ]:
# 아래 리스트를 삭제할 것임
list(retry1_df.loc[retry1_df['error'].notnull(), 'video_id'])

['aR4cXPWdTm0',
 'VU5UIyUTKrk',
 'GRHd6_gYzfw',
 '93d5yTcKzY0',
 '7ud88gf0P-8',
 '8lT4E63u2xg',
 'RvCSkwSMEOE',
 'wIlUEIWD_Iw',
 'zlbnEdIdlZg',
 'ThVzGu3-XPM',
 'J-M9khq5hLs',
 'aLdthnK_1UQ',
 'cFjAnSlYkmk',
 '3fQwMOb69o8',
 '29OWoBqY-eA',
 'RbUK3vkkR0U',
 'Ep-nD7IeHnM',
 '3XMcC__sHnQ',
 'DIaWb1ciHHY',
 'nQLDqRSiF_k',
 'fbxz-lu56NE',
 'urGtvYuMVYQ',
 'Vwk7wY_HtmI',
 '7KTgTqde-FE',
 'cKq36-nvC3E',
 '4huQ4Q7nRdM',
 '0TUPSJSMQdc',
 'XlnwgHSsSdA',
 'tTi67LVX3hU',
 'EM6Karmdf7s',
 'NMmr0ukHEGo',
 'D-FZ5OTw-NI',
 'rgAEPHxrwgA',
 'KsFcb9xs7eQ',
 'yfLYCxeobbU',
 'GU22ga_7D2s',
 'PWtMDf36wAA',
 'ULwLE6L-wL4',
 'iE_zT3vlDNE',
 'a_Pv8PmlDMY',
 'EJUy5prRQks',
 'faEIo-k2BZY',
 'AE11rWYdHdQ',
 '7cyheohi0YU',
 'ci_6i0T7o1M',
 'eGPuC9Z2riU',
 'oX3VdftQzLQ',
 'ajF2J-PzJWE',
 'T5B-zcm5UO0',
 'ucs7n403hPw',
 '84VFoLLqJn8',
 '6UoB6LVl0YY',
 'vnYV3Twx7lU',
 'M7tpUacxXyc',
 '8lfLS8ONB5g',
 'NEBIFMrJ7VI',
 'UZVgD0Nf9Sg',
 'Oa87_UgW57A',
 'VcdwvmV8sTQ',
 '4j80w4_I3fk',
 'p5YhWhpl0ZI',
 'Z8IvrKBPlew',
 'UxIMzy

In [ ]:
len(list(retry1_df.loc[retry1_df['error'].notnull(), 'video_id'])) # 87개

87

In [ ]:
# 기존 것을 오류 재처리한 것 넣어서 수정하기
pre_df = pd.read_csv('raw_data/audio_features/audio_features_with_errors.csv', encoding='utf-8')
pre_df

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
0,s6suc9Xz0rM,https://www.youtube.com/watch?v=s6suc9Xz0rM,79.6160,0.1234,1168.3195,[61.81318681],152.0,0.3627,NaN
1,kERWAGtltBU,https://www.youtube.com/watch?v=kERWAGtltBU,138.9813,0.1011,1351.3056,[83.95522388],124.0,0.5013,NaN
2,8VcAaEfuc0s,https://www.youtube.com/watch?v=8VcAaEfuc0s,89.3799,0.1323,862.8627,[108.17307692],176.0,0.3733,NaN
3,CI331OcJ8Do,https://www.youtube.com/watch?v=CI331OcJ8Do,176.4752,0.0737,1687.4316,[160.71428571],208.0,0.5013,NaN
4,AOrtPpF1t7U,https://www.youtube.com/watch?v=AOrtPpF1t7U,113.2603,0.0968,924.5371,[77.05479452],180.0,0.5547,NaN
...,...,...,...,...,...,...,...,...,...
6406,fLNAk4NYOJU,https://www.youtube.com/watch?v=fLNAk4NYOJU,91.7625,0.1175,1180.3476,[234.375],240.0,0.1600,NaN
6407,Sh41rnVFlhU,https://www.youtube.com/watch?v=Sh41rnVFlhU,109.4980,0.2931,2079.7808,[156.25],424.0,0.4693,NaN
6408,z1UGEkHIoCY,https://www.youtube.com/watch?v=z1UGEkHIoCY,103.9737,0.0451,343.8384,[119.68085106],60.0,0.3733,NaN
6409,xVaTzgcJttI,https://www.youtube.com/watch?v=xVaTzgcJttI,136.2584,0.1755,1891.7688,[144.23076923],188.0,1.2480,NaN


In [ ]:
# retry에서 error났던 video_id는 제외하고 pre_df에서 가져오기
filtered_df = pre_df.loc[~pre_df['video_id'].isin(retry_errors), :]
filtered_df

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
0,s6suc9Xz0rM,https://www.youtube.com/watch?v=s6suc9Xz0rM,79.6160,0.1234,1168.3195,[61.81318681],152.0,0.3627,NaN
1,kERWAGtltBU,https://www.youtube.com/watch?v=kERWAGtltBU,138.9813,0.1011,1351.3056,[83.95522388],124.0,0.5013,NaN
2,8VcAaEfuc0s,https://www.youtube.com/watch?v=8VcAaEfuc0s,89.3799,0.1323,862.8627,[108.17307692],176.0,0.3733,NaN
3,CI331OcJ8Do,https://www.youtube.com/watch?v=CI331OcJ8Do,176.4752,0.0737,1687.4316,[160.71428571],208.0,0.5013,NaN
4,AOrtPpF1t7U,https://www.youtube.com/watch?v=AOrtPpF1t7U,113.2603,0.0968,924.5371,[77.05479452],180.0,0.5547,NaN
...,...,...,...,...,...,...,...,...,...
6406,fLNAk4NYOJU,https://www.youtube.com/watch?v=fLNAk4NYOJU,91.7625,0.1175,1180.3476,[234.375],240.0,0.1600,NaN
6407,Sh41rnVFlhU,https://www.youtube.com/watch?v=Sh41rnVFlhU,109.4980,0.2931,2079.7808,[156.25],424.0,0.4693,NaN
6408,z1UGEkHIoCY,https://www.youtube.com/watch?v=z1UGEkHIoCY,103.9737,0.0451,343.8384,[119.68085106],60.0,0.3733,NaN
6409,xVaTzgcJttI,https://www.youtube.com/watch?v=xVaTzgcJttI,136.2584,0.1755,1891.7688,[144.23076923],188.0,1.2480,NaN


In [ ]:
# 1. retry1_df에서 오류 없는 행만 추출
retry_valid = retry1_df[retry1_df['video_id'].isin(retry_not_errors)]

# 2. 덮어쓸 컬럼 리스트
cols_to_update = ['pitch_mean', 'energy_mean', 'centroid_mean', 'bpm', 'speech_rate', 'initial_silence']

# 3. filtered_df에서 각 video_id에 대해 해당 값 덮어쓰기
for col in cols_to_update:
    filtered_df.loc[
        filtered_df['video_id'].isin(retry_valid['video_id']),
        col
    ] = filtered_df.loc[
        filtered_df['video_id'].isin(retry_valid['video_id']),
        'video_id'
    ].map(retry_valid.set_index('video_id')[col])
filtered_df

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
0,s6suc9Xz0rM,https://www.youtube.com/watch?v=s6suc9Xz0rM,79.6160,0.1234,1168.3195,[61.81318681],152.0,0.3627,NaN
1,kERWAGtltBU,https://www.youtube.com/watch?v=kERWAGtltBU,138.9813,0.1011,1351.3056,[83.95522388],124.0,0.5013,NaN
2,8VcAaEfuc0s,https://www.youtube.com/watch?v=8VcAaEfuc0s,89.3799,0.1323,862.8627,[108.17307692],176.0,0.3733,NaN
3,CI331OcJ8Do,https://www.youtube.com/watch?v=CI331OcJ8Do,176.4752,0.0737,1687.4316,[160.71428571],208.0,0.5013,NaN
4,AOrtPpF1t7U,https://www.youtube.com/watch?v=AOrtPpF1t7U,113.2603,0.0968,924.5371,[77.05479452],180.0,0.5547,NaN
...,...,...,...,...,...,...,...,...,...
6406,fLNAk4NYOJU,https://www.youtube.com/watch?v=fLNAk4NYOJU,91.7625,0.1175,1180.3476,[234.375],240.0,0.1600,NaN
6407,Sh41rnVFlhU,https://www.youtube.com/watch?v=Sh41rnVFlhU,109.4980,0.2931,2079.7808,[156.25],424.0,0.4693,NaN
6408,z1UGEkHIoCY,https://www.youtube.com/watch?v=z1UGEkHIoCY,103.9737,0.0451,343.8384,[119.68085106],60.0,0.3733,NaN
6409,xVaTzgcJttI,https://www.youtube.com/watch?v=xVaTzgcJttI,136.2584,0.1755,1891.7688,[144.23076923],188.0,1.2480,NaN


In [ ]:
# null 값 확인
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6324 entries, 0 to 6410
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   video_id         6324 non-null   object 
 1   url              6324 non-null   object 
 2   pitch_mean       6319 non-null   float64
 3   energy_mean      6324 non-null   float64
 4   centroid_mean    6324 non-null   float64
 5   bpm              6324 non-null   object 
 6   speech_rate      6324 non-null   float64
 7   initial_silence  6324 non-null   float64
 8   error            324 non-null    object 
dtypes: float64(5), object(4)
memory usage: 494.1+ KB


In [ ]:
filtered_df.loc[filtered_df['pitch_mean'].isnull(), :]

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
3332,q3FEbYsyy7M,https://www.youtube.com/watch?v=q3FEbYsyy7M,NaN,0.0000,0.0000,0.0000,0.0,15.0000,NaN
3333,2FnF1WzQx58,https://www.youtube.com/watch?v=2FnF1WzQx58,NaN,0.0000,0.0000,0.0000,0.0,15.0000,NaN
4365,J9DNUz52iY8,https://www.youtube.com/watch?v=J9DNUz52iY8,NaN,0.0925,4594.1872,[78.125],288.0,0.0213,NaN
5259,4W1-eIJ_cnQ,https://www.youtube.com/watch?v=4W1-eIJ_cnQ,NaN,0.0000,0.0000,0.0,0.0,15.0000,"Command '['yt-dlp', 'https://www.youtube.com/w..."
6160,olxg_Vbg7z0,https://www.youtube.com/watch?v=olxg_Vbg7z0,NaN,0.0000,0.0000,0.0000,0.0,15.0000,NaN


In [ ]:
final_filtered_df = filtered_df.loc[filtered_df['pitch_mean'].notnull(), :]
final_filtered_df

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
0,s6suc9Xz0rM,https://www.youtube.com/watch?v=s6suc9Xz0rM,79.6160,0.1234,1168.3195,[61.81318681],152.0,0.3627,NaN
1,kERWAGtltBU,https://www.youtube.com/watch?v=kERWAGtltBU,138.9813,0.1011,1351.3056,[83.95522388],124.0,0.5013,NaN
2,8VcAaEfuc0s,https://www.youtube.com/watch?v=8VcAaEfuc0s,89.3799,0.1323,862.8627,[108.17307692],176.0,0.3733,NaN
3,CI331OcJ8Do,https://www.youtube.com/watch?v=CI331OcJ8Do,176.4752,0.0737,1687.4316,[160.71428571],208.0,0.5013,NaN
4,AOrtPpF1t7U,https://www.youtube.com/watch?v=AOrtPpF1t7U,113.2603,0.0968,924.5371,[77.05479452],180.0,0.5547,NaN
...,...,...,...,...,...,...,...,...,...
6406,fLNAk4NYOJU,https://www.youtube.com/watch?v=fLNAk4NYOJU,91.7625,0.1175,1180.3476,[234.375],240.0,0.1600,NaN
6407,Sh41rnVFlhU,https://www.youtube.com/watch?v=Sh41rnVFlhU,109.4980,0.2931,2079.7808,[156.25],424.0,0.4693,NaN
6408,z1UGEkHIoCY,https://www.youtube.com/watch?v=z1UGEkHIoCY,103.9737,0.0451,343.8384,[119.68085106],60.0,0.3733,NaN
6409,xVaTzgcJttI,https://www.youtube.com/watch?v=xVaTzgcJttI,136.2584,0.1755,1891.7688,[144.23076923],188.0,1.2480,NaN


* 6319개 땅땅땅

In [ ]:
final_filtered_df['bpm'] = final_filtered_df['bpm'].apply(
    lambda x: round(ast.literal_eval(x)[0], 4) if isinstance(x, str) and x.startswith('[') else None
)

final_filtered_df.to_csv('raw_data/audio_features/final_audio_features.csv', encoding='utf-8', index=False)

### 최종 데이터프레임에서 87개 삭제 (YoutubeAPI 파일도 수정하는 코드)

In [ ]:
youtubeAPI_df = pd.read_csv("raw_data/youtube_info/비공개영상까지삭제(최종전)/final_all_youtubeAPI_info.csv", encoding="utf-8")
youtubeAPI_df

,video_idx,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"['재즈', '로', '무드있', '고', '잔잔하게', '내', '방', '근사하...",1.0,1.0
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"['일', '할', '때', '틀어', '놓기', '좋은', '업무', '용', '...",0.0,1.0
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"['여유롭게', '쉬', '고', '싶을', '때', '잔잔히', '틀기', '좋은...",1.0,1.0
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"['기분', '좋은', '아침', '여', '모닝', '겨울', '재즈', '하루'...",0.0,1.0
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","['재즈', '내리는', '계절', '겨울', '눈', '오는', '날', '듣기'...",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6406,6406,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,https://www.youtube.com/watch?v=fLNAk4NYOJU,2024-03-21T04:00:19Z,https://i.ytimg.com/vi/fLNAk4NYOJU/hqdefault.jpg,64905,790,27,NaN,fLNAk4NYOJU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,"['벚꽃', '만', '개', '하', '봄', '봄', '햇살', '함께', '듣...",0.0,1.0
6407,6407,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"['도쿄', '긴자', '밤', '닮은', '중독', '적', '인', '리듬', ...",1.0,0.0
6408,6408,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"['쓸쓸한', '도쿄', '퇴근길', '조용한', '밤', '듣기', '좋은', '...",1.0,1.0
6409,6409,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"['사랑', '하는', '너', '열차', '안', '함께', '노래', '듣고싶어...",1.0,0.0


In [ ]:
del youtubeAPI_df['video_idx']
youtubeAPI_df.head()

,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"['재즈', '로', '무드있', '고', '잔잔하게', '내', '방', '근사하...",1.0,1.0
1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"['일', '할', '때', '틀어', '놓기', '좋은', '업무', '용', '...",0.0,1.0
2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"['여유롭게', '쉬', '고', '싶을', '때', '잔잔히', '틀기', '좋은...",1.0,1.0
3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"['기분', '좋은', '아침', '여', '모닝', '겨울', '재즈', '하루'...",0.0,1.0
4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","['재즈', '내리는', '계절', '겨울', '눈', '오는', '날', '듣기'...",1.0,1.0


In [ ]:
# 오디오 때문에 삭제하고 남은 최종 데이터프레임
real_final_all_youtubeAPI_info = youtubeAPI_df.loc[~(youtubeAPI_df['video_id'].isin(
    ['aR4cXPWdTm0', 'VU5UIyUTKrk', 'GRHd6_gYzfw', '93d5yTcKzY0', '7ud88gf0P-8',
     '8lT4E63u2xg', 'RvCSkwSMEOE', 'wIlUEIWD_Iw', 'zlbnEdIdlZg', 'ThVzGu3-XPM',
     'J-M9khq5hLs', 'aLdthnK_1UQ', 'cFjAnSlYkmk', '3fQwMOb69o8', '29OWoBqY-eA',
     'RbUK3vkkR0U', 'Ep-nD7IeHnM', '3XMcC__sHnQ', 'DIaWb1ciHHY', 'nQLDqRSiF_k',
     'fbxz-lu56NE', 'urGtvYuMVYQ', 'Vwk7wY_HtmI', '7KTgTqde-FE', 'cKq36-nvC3E',
     '4huQ4Q7nRdM', '0TUPSJSMQdc', 'XlnwgHSsSdA', 'tTi67LVX3hU', 'EM6Karmdf7s',
     'NMmr0ukHEGo', 'D-FZ5OTw-NI', 'rgAEPHxrwgA', 'KsFcb9xs7eQ', 'yfLYCxeobbU',
     'GU22ga_7D2s', 'PWtMDf36wAA', 'ULwLE6L-wL4', 'iE_zT3vlDNE', 'a_Pv8PmlDMY',
     'EJUy5prRQks', 'faEIo-k2BZY', 'AE11rWYdHdQ', '7cyheohi0YU', 'ci_6i0T7o1M',
     'eGPuC9Z2riU', 'oX3VdftQzLQ', 'ajF2J-PzJWE', 'T5B-zcm5UO0', 'ucs7n403hPw',
     '84VFoLLqJn8', '6UoB6LVl0YY', 'vnYV3Twx7lU', 'M7tpUacxXyc', '8lfLS8ONB5g',
     'NEBIFMrJ7VI', 'UZVgD0Nf9Sg', 'Oa87_UgW57A', 'VcdwvmV8sTQ', '4j80w4_I3fk',
     'p5YhWhpl0ZI', 'Z8IvrKBPlew', 'UxIMzyPfnQ0', 'CXvDo7Sg800', 'JhNgb-yOxtE',
     'Qhe9OSA96uM', 'CfmTK-baSPU', '4hvWRsZJiag', '9CYxVsFu_dA', 'p4vqfrL_1ew',
     '2iV4-lS28tY', 'G-ie9Igk6qc', 'wBU1owueras', 'Y5wp8g1_Qlw', 'ZbNq-Pl5Rz8',
     'ezykDVItXFI', 'FT5RqC_S-xM', 'hejFoS8SlIQ', '0Q3nYDKOlF4', '_xjvovktwZw',
     'r40JmhK8Q0w', 'lhpYR74TCN0', 'bPRpGeFb4xM', 'zjwZ73U236I', 'JKBCcxRry-U',
     'Xc8nr_GQoZw', 'AbDJl2CTPLs', 'q3FEbYsyy7M', '2FnF1WzQx58', 'J9DNUz52iY8',
     '4W1-eIJ_cnQ', 'olxg_Vbg7z0'])),:]
real_final_all_youtubeAPI_info.reset_index(drop=True, inplace=True)
real_final_all_youtubeAPI_info.to_csv('raw_data/youtube_info/오디오없는것삭제(최종)/real_final_all_youtubeAPI_info.csv', index=False)

### 확인용 코드

In [ ]:
filtered_df.loc[filtered_df['error'].notnull(),:]

,video_id,url,pitch_mean,energy_mean,centroid_mean,bpm,speech_rate,initial_silence,error
29,Mzpga5zM3gI,https://www.youtube.com/watch?v=Mzpga5zM3gI,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
32,9vunx_Fzzdc,https://www.youtube.com/watch?v=9vunx_Fzzdc,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
63,CbaOrIR20Uo,https://www.youtube.com/watch?v=CbaOrIR20Uo,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
113,IcHeRHYBd6I,https://www.youtube.com/watch?v=IcHeRHYBd6I,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
138,oOQ7Ahe8vmI,https://www.youtube.com/watch?v=oOQ7Ahe8vmI,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
...,...,...,...,...,...,...,...,...,...
6336,kXEB4zrdg9g,https://www.youtube.com/watch?v=kXEB4zrdg9g,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
6352,NXNOK_-IilY,https://www.youtube.com/watch?v=NXNOK_-IilY,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
6371,gAtOZTt8-BM,https://www.youtube.com/watch?v=gAtOZTt8-BM,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."
6383,OvodpGjSna8,https://www.youtube.com/watch?v=OvodpGjSna8,NaN,NaN,NaN,NaN,NaN,NaN,"Command '['yt-dlp', 'https://www.youtube.com/w..."


In [ ]:
retry_not_errors = retry1_df.loc[retry1_df['error'].isnull(), 'video_id']
retry_not_errors

0      Mzpga5zM3gI
1      9vunx_Fzzdc
2      CbaOrIR20Uo
3      IcHeRHYBd6I
4      oOQ7Ahe8vmI
          ...     
405    kXEB4zrdg9g
406    NXNOK_-IilY
407    gAtOZTt8-BM
408    OvodpGjSna8
410    WEkYWyi1gfA
Name: video_id, Length: 324, dtype: object

In [ ]:
6411-6324

87

In [ ]:
# 확인용
set(filtered_df.loc[filtered_df['error'].notnull(),'video_id']) == set(retry_not_errors)

True